In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터

2016년 12월 14일부터 2018년 7월 30일까지 train data

2016년 8월 1일부터 2018년 3월 31일까지 test data

store_id : 각 파일에서의 상점 고유 번호 (해당기한 내 개업 혹은 폐업한 상점도 있음)

date : 거래일자

time : 거래 시간

card_id : 카드 번호의 hash 값

amount : 매출 금액, 음수인 경우 취소 거래

installments : 할부 개월 수. 일시불은 빈 문자열

days_of_week : 요일, 월요일이 0, 일요일은 6

holiday : 1이면 공휴일, 0이면 공휴일 아님

total_sales ; 해당 기간의 총 sales (예측)

**test.csv와 train.csv store_id 같아도 같은 상점이 아님** 

In [2]:
#기본 라이브러리 설치

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#display
from IPython.display import display

#진행상황
from tqdm import tqdm

#모델 
import xgboost as xgb

In [19]:
train_data = pd.read_csv('/content/drive/MyDrive/대출 상점/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/대출 상점/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/대출 상점/submission.csv')

merge_df = pd.concat([train_data, test_data]) # train, test 데이터 합치기

train_data.drop('installments', axis=1, inplace=True)
test_data.drop('installments', axis=1, inplace=True)

In [20]:
merge_df.head(10)

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0
5,0,2016-12-19,13:36:39,30b1422f77,44,NaN,0,0
6,0,2016-12-19,14:00:23,6dd8dad5ab,146,NaN,0,0
7,0,2016-12-19,14:01:36,a4a425c5d2,6,NaN,0,0
8,0,2016-12-19,14:09:46,f213c011fe,306,NaN,0,0
9,0,2016-12-19,14:13:02,a185b7b11f,51,NaN,0,0


# 전체 날짜의 휴일 요일

In [21]:
# 원본 부분 바꾼 거 아님 data 부분만 빼와서 한 거 같은데

all_dates = merge_df.drop_duplicates('date').sort_values('date') # 날짜 중복되는 거 제거하고, 정렬
all_dates = all_dates[['date', 'days_of_week', 'holyday']]
all_dates.reset_index(drop=True, inplace = True) # 기본값으로 인덱스 값들을 DataFrame의 열로 전송, 새로운 정수 인덱스 세팅
# drop -> 인덱스로 세팅한 열을 DataFrame 내에서 삭제할 지
# inplace -> 원본 객체를 변경할 지 
all_dates['date'] = pd.to_datetime(all_dates['date'])

In [22]:
holidays = all_dates[all_dates['holyday']>0][['date', 'holyday']] # holyday==1인 날짜
holidays.columns=['ds', 'holiday'] #열 이름 바꿔줌
holidays['lower_window'] = 0 # 열 추가 -> 왜?
holidays['upper_window'] = 0 # 열 추가 -> 왜?

holidays.holiday = holidays.holiday.astype('str')

날짜 특성 변수 설정


In [23]:
train_data['date'] = train_data['date'] + ' ' + train_data['time']
train_data['date'] = pd.to_datetime(train_data['date'])
train_data.drop('time', axis=1)
test_data['date'] = test_data['date'] + ' ' + test_data['time']
test_data['date'] = pd.to_datetime(test_data['date'])
test_data.drop('time', axis=1)

,store_id,date,card_id,amount,days_of_week,holyday
0,0,2016-08-01 00:28:15,bf33518373,125,0,0
1,0,2016-08-01 01:09:58,7a19a3a92f,90,0,0
2,0,2016-08-01 01:47:24,6f9fd7e241,150,0,0
3,0,2016-08-01 17:54:43,8bcf1d61b2,362,0,0
4,0,2016-08-01 18:48:53,6a722ce674,125,0,0
...,...,...,...,...,...,...
473387,199,2018-03-30 14:17:59,300d7bc922,65,4,0
473388,199,2018-03-30 19:01:54,3ab757718b,65,4,0
473389,199,2018-03-30 20:08:03,2d8e9e421c,65,4,0
473390,199,2018-03-30 20:11:58,22daeb334e,200,4,0


In [26]:
train_data['day'] = train_data['date'].dt.day  #pandas 시간 함수
train_data['hour'] = train_data['date'].dt.hour
train_data['year'] = train_data['date'].dt.year
train_data['month'] = train_data['date'].dt.month
train_data['minute'] = train_data['date'].dt.minute

test_data['day'] = test_data['date'].dt.day
test_data['hour'] = test_data['date'].dt.hour
test_data['year'] = test_data['date'].dt.year
test_data['month'] = test_data['date'].dt.month
test_data['minute'] = test_data['date'].dt.minute

In [28]:
display(train_data)
display(test_data)

,store_id,date,time,card_id,amount,days_of_week,holyday,day,hour,year,month,minute
0,0,2016-12-14 18:05:31,18:05:31,d297bba73f,5,2,0,14,18,2016,12,5
1,0,2016-12-14 18:05:54,18:05:54,d297bba73f,-5,2,0,14,18,2016,12,5
2,0,2016-12-19 12:42:31,12:42:31,0880849c05,144,0,0,19,12,2016,12,42
3,0,2016-12-19 12:48:08,12:48:08,8b4f9e0e95,66,0,0,19,12,2016,12,48
4,0,2016-12-19 13:31:08,13:31:08,7ad237eed0,24,0,0,19,13,2016,12,31
...,...,...,...,...,...,...,...,...,...,...,...,...
3362791,1799,2018-07-19 17:58:31,17:58:31,e254bf70d9,600,3,0,19,17,2018,7,58
3362792,1799,2018-07-19 18:54:34,18:54:34,8f41c89891,275,3,0,19,18,2018,7,54
3362793,1799,2018-07-22 14:46:57,14:46:57,aeb64fe1fb,350,6,0,22,14,2018,7,46
3362794,1799,2018-07-25 18:09:13,18:09:13,57932602d6,300,2,0,25,18,2018,7,9


,store_id,date,time,card_id,amount,days_of_week,holyday,day,hour,year,month,minute
0,0,2016-08-01 00:28:15,00:28:15,bf33518373,125,0,0,1,0,2016,8,28
1,0,2016-08-01 01:09:58,01:09:58,7a19a3a92f,90,0,0,1,1,2016,8,9
2,0,2016-08-01 01:47:24,01:47:24,6f9fd7e241,150,0,0,1,1,2016,8,47
3,0,2016-08-01 17:54:43,17:54:43,8bcf1d61b2,362,0,0,1,17,2016,8,54
4,0,2016-08-01 18:48:53,18:48:53,6a722ce674,125,0,0,1,18,2016,8,48
...,...,...,...,...,...,...,...,...,...,...,...,...
473387,199,2018-03-30 14:17:59,14:17:59,300d7bc922,65,4,0,30,14,2018,3,17
473388,199,2018-03-30 19:01:54,19:01:54,3ab757718b,65,4,0,30,19,2018,3,1
473389,199,2018-03-30 20:08:03,20:08:03,2d8e9e421c,65,4,0,30,20,2018,3,8
473390,199,2018-03-30 20:11:58,20:11:58,22daeb334e,200,4,0,30,20,2018,3,11
